In [ ]:
import binascii
import base64
import requests
import json
import requests

from datetime import datetime
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec

### Parameter

In [ ]:
app_secret = 'YOUR APP SECRET HERE' #<<<<<<<<<<<<<<<<<<<<<<<<<<<INPUT
app_id = 'YOUR APP ID HERE' #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<INPUT

broker_id = '063' #Classic Ausiris
account_id = 'YOUR ACCOUNT ID HERE' #<<<<<<<<<<<<<<<<<<<<<<<<<<INPUT

system_id = 'DERIV' #Derivatives

### Endpoint

In [ ]:
#Authentication Management
url_auth_login = 'https://open-api.settrade.com/api/oam/v1/{:}/broker-apps/ALGO/login'.format(broker_id)
url_auth_refresh = 'https://open-api.settrade.com/api/oam/v1/{:}/broker-apps/ALGO/refresh-token'.format(broker_id)
#User Management API
url_usr_currentuser = 'http://open-api.settrade.com/api/um/v1/{:}/user/me'.format(broker_id)
url_usr_defaultuser = 'http://open-api.settrade.com/api/um/v1/{:}/user/accounts/{:}/default'.format(broker_id, system_id)
url_usr_role = 'http://open-api.settrade.com/api/um/v1/{:}/user/roles/{:}'.format(broker_id, system_id)
url_usr_traderid = 'http://open-api.settrade.com/api/um/v1/{:}/user/trader-id/{:}'.format(broker_id, system_id)
url_usr_listacc = 'http://open-api.settrade.com/api/um/v1/{:}/user/accounts/{:}'.format(broker_id, system_id)
url_usr_verify = 'https://open-api.settrade.com/api/um/v1/{:}/user/verify-pin'.format(broker_id)
#Derivatives API
url_deri_info = 'http://open-api.settrade.com/api/seosd/v1/{:}/accounts/{:}/account-info'.format(broker_id, account_id)

In [ ]:
#Token keeper
token = {}

In [ ]:
#Encryption function to get a signature
def get_signature(app_secret, app_id, params=''):
    #Get timestamp in millisecond and round
    timestamp = round(datetime.timestamp(datetime.now())*1000)
    #Combind requested content.
    content = '{:}.{:}.{:}'.format(app_id, params, timestamp)
    #Convert secret key form base64 to integer.
    secret_key = int.from_bytes(base64.b64decode(app_secret), 'big')
    #Create Private key using SECP256R1 algorithm with secret key.
    private_key = ec.derive_private_key(secret_key, ec.SECP256R1(), default_backend())
    #Sign content with private key and hashes to SHA256
    signature = private_key.sign(content.encode(),ec.ECDSA(hashes.SHA256()))
    #Return timestamp, string signature that converted from HEX 
    return timestamp, binascii.hexlify(signature).decode()

### Get Token

In [ ]:
params = '' #Currently, params should be empty.
timestamp, signature = get_signature(app_secret, app_id, params=params)

#Required header, you can't run without this.
headers= { 'Host': 'open-api.settrade.com' }
#Required Payload in Json format.
payload = {
    "apiKey": app_id,
    "params": params,
    "signature": signature,
    "timestamp": timestamp
}
#Post request to endpoint.
response = requests.post(url_auth_login, headers=headers, json=payload)
#If success, response code must be 200. Otherwise, You got the problem.
if response.status_code == 200:
     #Keep token to global.
    token = response.json()
    #Just for beautify display, in production can use only response.json()
    print(json.dumps(json.loads(response.text), indent=2)) 
else:
    print('Error Code =', response.status_code)

### Refresh Token

In [ ]:
headers= { 'Host': 'open-api.settrade.com' }
payload = {
    "apiKey": app_id,
    "refreshToken": token['refresh_token'] #Refresh token is located in 'token' variable.
}

response = requests.post(url_auth_refresh, headers=headers, json=payload)

if response.status_code == 200:
    token = response.json()
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### Get Current User

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token'] #Don't forget, access token must be use.
}
response = requests.get(url_usr_currentuser, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### Get Default Account By System

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}
response = requests.get(url_usr_defaultuser, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### Get Role

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}
response = requests.get(url_usr_role, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### Get Trader ID

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}
response = requests.get(url_usr_traderid, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### List Accounts

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}
response = requests.get(url_usr_listacc, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)

### Verify PIN

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}

payload = {
    'PIN': 'YOUR PIN CODE', #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<INPUT
    'systemId': 'DERIV' #Just for Derivatives
}

response = requests.post(url_usr_verify, headers=headers, json=payload)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print(response.text, 'Error Code =', response.status_code)

### Get  Account Info

In [ ]:
headers= {
    'Host': 'open-api.settrade.com',
    'Authorization': 'Bearer ' + token['access_token']
}
response = requests.get(url_deri_info, headers=headers)

if response.status_code == 200:
    print(json.dumps(json.loads(response.text), indent=2))
else:
    print('Error Code =', response.status_code)